In [ ]:
pip install webdriver_manager
# Hay que tener instalado webdriver_manager y selenium

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from selenium import webdriver
import operator
import re
import time
import pandas as pd
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime,timedelta
import time
import random
import re
import operator
#Impirtamos librerías

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())
# Corremos "webdriver". No abrirá una segunda página web donde correrá el resto del programa



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/raul.castillo/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


In [ ]:
url = "https://www.google.com/maps/place/UNITEC+Los+Reyes+-+Universidad+Tecnol%C3%B3gica+de+M%C3%A9xico/@19.3592234,-98.9935063,21z/data=!4m7!3m6!1s0x85d1e29828d3e631:0x324b294da57a4bf9!8m2!3d19.3591634!4d-98.9936022!9m1!1b1"
# 2 puntos relevantes en la URL. Debe estar dentro de reseñas y con un zoom a 21z  

In [ ]:
driver.get(url)
#Nos abre la URL en webdriver

In [ ]:
lugar = url[len('https://www.google.com.mx/maps/place/'):len('https://www.google.com.mx/maps/place/')+url[len('https://www.google.com.mx/maps/place/'):].find('/')].replace('+',' ')
lugar
# Extraemos el nombre del place de la URL

'TEC Los Reyes - Universidad Tecnol%C3%B3gica de M%C3%A9xico'

In [ ]:
lugar = 'UNITEC Los Reyes'
# Si el nombre que nos otorga la URL no es el que queremos conservar, podemos cambiarlo. De lo contrario, se ignora esta sección

In [ ]:
coord = url[url[:].find('@')+1 : url[:].find(',2')]
coord
#Obtenemos las coordenadas de la URL

'19.3592234,-98.9935063'

In [ ]:
y = coord[-coord.find(',')-1:]
y
#Obtenemos la coordenada "Y" por separado

'-98.9935063'

In [ ]:
x = coord[:-coord.find(',')-2]
x
#Obtenemos la coordenada "X" por separado

'19.3592234'

In [ ]:
html = driver.find_element_by_tag_name('html')
# Encuentra los elementos del código HTML

down = range(1,700)
for d in down:
    html.send_keys(Keys.PAGE_DOWN)
    ran = random.randint(6, 10)
    time.sleep(ran)
    #print(d, ran)

# Este es nuestro Robot para scrapear. El rango inicial es el número de veces que hará scroll, con una pausa de 6 a 10 segundo

In [ ]:
elem = driver.find_element_by_xpath("//*")
soup = BeautifulSoup(elem.get_attribute('innerHTML'),'html.parser')

# toma el código fuente de la página html 

In [ ]:
reviews = soup.find_all(("div",'jJc9Ad'))[0].get_text()
# encuentra el divisor que buscamos

In [ ]:
reviews = soup.find_all("div",'jJc9Ad')
no = range(0,len(reviews))
now = datetime.now() #- timedelta(hours=6) 
#hora = now.strftime('%H:%M')
date = now.strftime('%Y-%m-%d')
print (no)
# Nos indica con un rango, el número de reseñas que tenemos

range(0, 120)


In [ ]:
df = pd.DataFrame(columns=('name','gender','strellas','resena','fecha','lugar','proyecto','url','x','y'))
#Generamos la estructura de nuestro Data Frame

#df = pd.DataFrame(columns=('name','gender','strellas','resena','opiniones','fecha','lugar','proyecto','url','x','y'))
#dfi = pd.DataFrame(columns=('name','x','y','img','foto'))

In [ ]:
# Limpiamos texto y obtenemos el género de los usuairos. Tenemos que tener cargado el archivo nombres.csv en la misma carpeta donde está nuestrop script

def clean_text(txt):
    txt = re.sub("[^a-záéíóúñüäë]", " ", txt.lower())
    # Nos pasa el textoo a minúsculas.
    txt = re.sub(' +',' ', txt)
    # Limpia nuestro texto.
    return txt.strip().split()
    # Nos regresa el texto con las delimitaciones especificadas.

def df_to_dict(df, key_column, val_column):
    """convierte dos pandas series en un diccionario"""
    xkey = df[key_column].tolist()
    xval = df[val_column].tolist()
    return dict(zip(xkey,xval))

def get_gender2(names):
    names = clean_text(names)
    names = [x for x in names if gender_list.get(x,'a') != 'a']
    gender ={'m':0, 'f':0, 'a':0}
    for i, name in enumerate(names):
        g = gender_list.get(name,'a')
        gender[g] += 1
        gender[g] += 2 if len(names) > 1 and i == 0 and g != 'a' else 0 
    gender['a'] = 0 if (gender['f']+gender['m']) > 0 else 1
    return max(gender.items(), key=operator.itemgetter(1))[0]

gender_list = pd.read_csv('nombres.csv')
gender_list = df_to_dict(gender_list, key_column='nombre', val_column='genero')

In [ ]:
#Identificamos los fragmentos del código HTML de página y extraemos la información
for n in no:
    try:
        stars = reviews[n].find_all("span",'kvMYJc')[0]['aria-label']
        st = stars[1:2]
        comm = reviews[n].find_all("span",'wiI7pd')
        co = comm[0].get_text().replace('\n',' ').strip()
        names = reviews[n].find_all("div",'d4r55')
        na = names[0].text.strip()
        gen = get_gender2(na)

        # Obtenemos imágenes y sus URL
        for z in range(0,len(reviews[n].find_all("button",'ODSEW-ShBeI-xJzy8c')),1):
            imgs = reviews[n].find_all("button",'KtCyie')[z]['style']
            img = imgs[imgs.find('url(')+len('url('):imgs.find(');width')]
            #print(url)
            dfi = dfi.append({'name': na,'img':img,'x':x,'y':y,'foto':z},ignore_index=True)

        # La fehca hay que transformala por el formato en el que viene
        tiempo = reviews[0].find_all("span",'rsqaWe')[0].text    
        if 'una semana' in tiempo:
            now = datetime.now() - timedelta(weeks=1) 
            date = now.strftime('%Y-%m-%d')
            #print(n,tiempo, date)
        if 'semanas' in tiempo:
            now = datetime.now() - timedelta(weeks=int(tiempo[:1])) 
            date = now.strftime('%Y-%m-%d')
            #print(n,tiempo, date)
        if 'minutos' in tiempo:
            now = datetime.now()
            date = now.strftime('%Y-%m-%d')
            #print(n,tiempo, date)
        if 'horas' in tiempo:
            now = datetime.now()
            date = now.strftime('%Y-%m-%d')
            #print(n,tiempo, date)
        if 'día' in tiempo:
            now = datetime.now() - timedelta(days=int(tiempo[len('hace '):6])) 
            date = now.strftime('%Y-%m-%d')
            #print(n,tiempo, date)
        if 'un mes' in tiempo:
            now = datetime.now() - timedelta(weeks=4*1) 
            date = now.strftime('%Y-%m-%d')
            #print(n,tiempo, date)
        if 'meses' in tiempo:
            now = datetime.now() - timedelta(weeks=4*int(tiempo[len('hace '):len(' meses')+1])) 
            date = now.strftime('%Y-%m-%d')
            #print(n,tiempo, date)
        if 'años' in tiempo:
            now = datetime.now() - timedelta(weeks=4*int(tiempo[:1])*12) 
            date = now.strftime('%Y-%m-%d')
                #print(n,tiempo, date)

    except:
        pass
    
    # Generamos nuestro Data Frame Final
    df = df.append({'name': na,'gender':gen,'strellas':st,'resena':co,
                    'fecha':date,'lugar':lugar,'proyecto':'TecMilenio','url':url,'x':x,'y':y},ignore_index=True)

In [ ]:
df

,name,gender,strellas,resena,fecha,lugar,proyecto,url,x,y
0,Olivia Ferreira,f,5,Ya me dieron seguimiento con mucha amabilidad ...,2022-03-16,UNITEC Los Reyes,TecMilenio,https://www.google.com/maps/place/UNITEC+Los+R...,19.3592234,-98.9935063
1,David Alva,m,1,"Pésimo servicio educativo, administrativo, pur...",2022-03-16,UNITEC Los Reyes,TecMilenio,https://www.google.com/maps/place/UNITEC+Los+R...,19.3592234,-98.9935063
2,ᴢᴜʀɪᴇʟ ᴊᴏꜱᴜᴇ ᴠᴀʟᴀᴅᴇᴢ,a,1,"Pesimo servicio, y que niveo educativo tan feo...",2022-03-16,UNITEC Los Reyes,TecMilenio,https://www.google.com/maps/place/UNITEC+Los+R...,19.3592234,-98.9935063
3,cristian,m,1,Da mucho miedo la zona y bueno la educación ah...,2022-03-16,UNITEC Los Reyes,TecMilenio,https://www.google.com/maps/place/UNITEC+Los+R...,19.3592234,-98.9935063
4,M Gm,a,1,Yo solo fui a pedir informes y según yo ya est...,2022-03-16,UNITEC Los Reyes,TecMilenio,https://www.google.com/maps/place/UNITEC+Los+R...,19.3592234,-98.9935063
...,...,...,...,...,...,...,...,...,...,...
115,Vega Ruiz Ximena Itzel,f,5,,2022-03-16,UNITEC Los Reyes,TecMilenio,https://www.google.com/maps/place/UNITEC+Los+R...,19.3592234,-98.9935063
116,Adrian Portillo,m,5,,2022-03-16,UNITEC Los Reyes,TecMilenio,https://www.google.com/maps/place/UNITEC+Los+R...,19.3592234,-98.9935063
117,Raúl López,m,5,,2022-03-16,UNITEC Los Reyes,TecMilenio,https://www.google.com/maps/place/UNITEC+Los+R...,19.3592234,-98.9935063
118,Jessy Molina,f,5,,2022-03-16,UNITEC Los Reyes,TecMilenio,https://www.google.com/maps/place/UNITEC+Los+R...,19.3592234,-98.9935063


In [ ]:
df.groupby(['x']).count()

,name,gender,strellas,resena,fecha,lugar,proyecto,url,y
x,,,,,,,,,
19.3592234,120,120,120,120,120,120,120,120,120


In [ ]:
export_csv = df.to_csv ("TM_UNITEC_10.csv", index = None, header=True , encoding='UTF-8')

In [ ]:
#pip install plotly

In [ ]:
import plotly.express as px
fig = px.density_mapbox(df, lat='x', lon='y', z='strellas', radius=5,
                        center=dict(lat=20.69, lon=-100.43), zoom=8,
                        mapbox_style="open-street-map")
fig.show()

In [ ]:
pwd

'/Users/raul.castillo/SCRIPTS/GOOGLE Places'

In [ ]:
df.columns

Index(['name', 'gender', 'strellas', 'resena', 'fecha', 'lugar', 'proyecto',
       'url', 'x', 'y'],
      dtype='object')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=851a6ba8-4407-4727-a931-5d5f428cca42' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>